See readme.md for ideal fields and descriptions

In [1]:
# Can do installs here when conde env is activated
# ! pip install pandas pyarrow
# ! pip install polars
# ! pip install maturin
# ! pip install cryo
# ! pip install web3

In [2]:
import os
import glob
from datetime import datetime, timedelta, timezone
import time
import pyarrow.parquet as pq
import pandas as pd
import cryo
import sys
sys.path.append("../../helper_functions")
import web3py_utils as w3py
import os_utils as osu
sys.path.pop()
import polars as pl
# test adding cryo_cli python

/Users/michaelsilberling/opt/anaconda3/envs/cryo-env/lib/python3.8/site-packages/eth_utils/network.py:61: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()
/Users/michaelsilberling/opt/anaconda3/envs/cryo-env/lib/python3.8/site-packages/eth_utils/network.py:61: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()


In [3]:
output_directory = 'cryo_outputs/'
# clear out
osu.clear_folder(output_directory)

Folder cleared.


In [4]:

redstone = {
        'rpc_url': 'https://rpc.redstonechain.com',
        'chain_name': 'redstone',
        'block_time_sec': 2,
        'block_time_buffer': 0,
        'stack': 'op'
        }
# Lyra
lyra = {
        'rpc_url': 'https://rpc.lyra.finance/',
        'chain_name': 'lyra',
        'block_time_sec': 2,
        'block_time_buffer': 0,
        'stack': 'op'
        }
degen = {
        'rpc_url': 'https://rpc.degen.tips',
        'chain_name': 'degen',
        'block_time_sec': 0.383, #Note: Arb Stack not deterministic
        'block_time_buffer': 0.25,
        'stack': 'arb'
        }




In [5]:
#Pick Chain
config_chain = degen
datasets = ['txs', 'logs', 'blocks']
###
whole_day_only = True
trailing_days = 0.25
dry_run = False
requests_per_second_max = 500 # -1 means ignore

In [6]:
#Intermediate Calc
rpc_url = config_chain['rpc_url']
stack = config_chain['stack']
chain_name = config_chain['chain_name']
# block_time_sec = config_chain['block_time_sec']
# block_time_buffer = config_chain['block_time_buffer']

# blocks_per_day = (60*60*24) / block_time_sec
# blocks_per_day_lo = (60*60*24) / (block_time_sec+block_time_buffer)
# if block_time_sec-block_time_buffer > 0:
#         blocks_per_day_hi = (60*60*24) / (block_time_sec-block_time_buffer)
# else:
#         blocks_per_day_hi = (60*60*24) / 0.01

# print(blocks_per_day)

In [7]:
# Init timestamps

# Get the current time in UTC
current_time_utc = datetime.utcnow()
# print(current_time_utc)

# If only whole days then shift the ending time to the start of the day
if whole_day_only:
        current_date_utc = datetime.combine(datetime.utcnow().date(), datetime.min.time())
        time_difference = current_time_utc - current_date_utc
        difference_days_fraction = time_difference.total_seconds() / (24 * 3600)  # There are 86400 seconds in a day
else: 
        current_date_utc = current_time_utc
        difference_days_fraction = 0
print('day fraction :' + str(difference_days_fraction))

print(current_date_utc)
starting_date_utc = current_date_utc - timedelta(days=trailing_days)

current_block = w3py.getLatestBlockNumber(rpc_url)

# ending_block = int( current_block - (difference_days_fraction * blocks_per_day_lo) )
# starting_block = int( ending_block - (trailing_days * blocks_per_day_hi) )

# print('current: ' + str(int(current_block)))
# print('end: ' + str(int(ending_block)))
# print('start: ' + str(int(starting_block)))

day fraction :0.01085538349537037
2024-05-10 00:00:00


In [8]:
print(starting_date_utc)
print(current_date_utc)
start_timestamp = int(starting_date_utc.timestamp())
end_timestamp = int(current_date_utc.timestamp())

start_time = time.time()

2024-05-09 18:00:00
2024-05-10 00:00:00


In [9]:
# Fetch and save data in JSON
data = cryo.freeze(
    datasets,
    # blocks=[str(starting_block) + ":" + str(ending_block)],
    timestamps=[str(start_timestamp) + ":" + str(end_timestamp)],
    rpc=rpc_url,
    output_dir= output_directory,
    file_format='parquet',
    label=chain_name,
    hex=True,
    dry=dry_run,
    requests_per_second=requests_per_second_max
)

cryo parameters
───────────────
- version: 152a635
- data: 
    - datatypes: blocks, transactions, logs
    - blocks: n=31,531 min=14,551,835 max=14,583,365 align=no reorg_buffer=0
- source: 
    - network: network_666666666
    - rpc url: https://rpc.degen.tips
    - max requests per second: 500
    - max concurrent requests: 500
    - max concurrent chunks: 4
    - inner request size: 1
- output: 
    - chunk size: 1,000
    - chunks to collect: 64 / 64
    - output format: parquet
    - output dir: /Users/michaelsilberling/Documents/GitHub/op-analytics/op_chains_tracking/lightweight_chain_tracker/cryo_outputs
    - report file: $OUTPUT_DIR/.cryo/reports/2024-05-09_20-15-43.063478.json


schema for blocks
─────────────────
- block_number: uint32
- block_hash: hex
- timestamp: uint32
- author: hex
- gas_used: uint64
- extra_data: hex
- base_fee_per_gas: uint64
- chain_id: uint64

sorting blocks by: block_number

other available columns: parent_hash, state_root, transactions_root, rece

In [10]:
# Calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Elapsed time: 853.4164 seconds


In [11]:
# Move this to a cryo_utils eventually

def load_parquet_files(chain_name, data_type_name, data_directory):
    pattern = f"*{data_type_name}__{chain_name}*.parquet"
    file_paths = glob.glob(os.path.join(data_directory, pattern))
    
    if file_paths:
        df = pl.scan_parquet(file_paths)
        # Further processing can go here, for example:
        # df = df.filter(pl.col("some_column") > 0)
        return df.collect()  # Collecting after all transformations
    else:
        print("No files found matching the pattern.")
        return None

In [12]:
# # Read parquet files
blocks = load_parquet_files(chain_name, 'blocks', output_directory)
txs = load_parquet_files(chain_name, 'transactions', output_directory)
logs = load_parquet_files(chain_name, 'logs', output_directory)

# Rename the 'gas_used' column to 'block_gas_used' in the 'blocks' DataFrame
blocks = blocks.rename({"gas_used": "block_gas_used"})
print(txs.columns)
print(blocks.columns)
print(logs.columns)

['block_number', 'transaction_index', 'transaction_hash', 'nonce', 'from_address', 'to_address', 'value_binary', 'value_string', 'value_f64', 'input', 'gas_limit', 'gas_used', 'gas_price', 'transaction_type', 'max_priority_fee_per_gas', 'max_fee_per_gas', 'success', 'n_input_bytes', 'n_input_zero_bytes', 'n_input_nonzero_bytes', 'chain_id']
['block_hash', 'author', 'block_number', 'block_gas_used', 'extra_data', 'timestamp', 'base_fee_per_gas', 'chain_id']
['block_number', 'transaction_index', 'log_index', 'transaction_hash', 'address', 'topic0', 'topic1', 'topic2', 'topic3', 'data', 'chain_id']


In [13]:
# Read the exclude_topics from CSV
non_app_methods_df = pl.read_csv("../inputs/non_app_methods.csv")
# print(non_app_methods_df)
exclude_topics = non_app_methods_df.filter(non_app_methods_df['type'] == "topic0")["method_id"].to_list()
# print(exclude_topics)

In [14]:

# Group by block_number, transaction_hash, and chain_id and count rows
logs_agg = (
    logs.group_by([pl.col("block_number"), pl.col("transaction_hash"), pl.col("chain_id")])
    .agg(
        num_event_logs = pl.col("topic0").count(),
        num_app_event_logs = pl.col("topic0").filter(~pl.col("topic0").is_in(exclude_topics)).count()
    )
)
# logs_agg[['transaction_hash','num_event_logs','num_app_event_logs']].glimpse()
# logs_agg.filter(pl.col("num_event_logs") != pl.col("num_app_event_logs")).glimpse()


In [15]:

# Perform the join on 'block_number' and 'chain_id'
joined_df = blocks.join(
    txs,
    on=["block_number", "chain_id"],
    how="inner"  # You can specify the type of join you want (inner, outer, left, right)
)
joined_df = joined_df.join(
    logs_agg,
    on=["block_number", "chain_id", "transaction_hash"],
    how="left"  # You can specify the type of join you want (inner, outer, left, right)
)
# Convert Unix timestamp to datetime and create a new column 'timestamp_dt'
joined_df = joined_df.with_columns(
    pl.from_epoch("timestamp", time_unit="s").alias("timestamp_dt")
)

# Truncate the 'timestamp_dt' column to the day and create a new column 'timestamp_date'
joined_df = joined_df.with_columns(
    pl.col("timestamp_dt").dt.truncate("1d").alias("timestamp_date")
)

In [16]:
print(joined_df['timestamp'].max())
print(joined_df['timestamp'].min())

print(joined_df.columns)
print(start_timestamp)
print(end_timestamp)
print(joined_df[['timestamp','timestamp_dt']])

1715300140
1715292000
['block_hash', 'author', 'block_number', 'block_gas_used', 'extra_data', 'timestamp', 'base_fee_per_gas', 'chain_id', 'transaction_index', 'transaction_hash', 'nonce', 'from_address', 'to_address', 'value_binary', 'value_string', 'value_f64', 'input', 'gas_limit', 'gas_used', 'gas_price', 'transaction_type', 'max_priority_fee_per_gas', 'max_fee_per_gas', 'success', 'n_input_bytes', 'n_input_zero_bytes', 'n_input_nonzero_bytes', 'num_event_logs', 'num_app_event_logs', 'timestamp_dt', 'timestamp_date']
1715292000
1715313600
shape: (292_441, 2)
┌────────────┬─────────────────────┐
│ timestamp  ┆ timestamp_dt        │
│ ---        ┆ ---                 │
│ u32        ┆ datetime[μs]        │
╞════════════╪═════════════════════╡
│ 1715293802 ┆ 2024-05-09 22:30:02 │
│ 1715293802 ┆ 2024-05-09 22:30:02 │
│ 1715293802 ┆ 2024-05-09 22:30:02 │
│ 1715293802 ┆ 2024-05-09 22:30:02 │
│ 1715293802 ┆ 2024-05-09 22:30:02 │
│ …          ┆ …                   │
│ 1715300140 ┆ 2024-05-

In [17]:
# print(blocks.schema)
# print(txs.schema)
print(joined_df.schema)
print(type(joined_df))

#test output
joined_pd = joined_df.to_pandas()
# print(joined_pd.tail(5))

print('num blocks: ' + str(joined_pd['block_number'].nunique()))

OrderedDict([('block_hash', String), ('author', String), ('block_number', UInt32), ('block_gas_used', UInt64), ('extra_data', String), ('timestamp', UInt32), ('base_fee_per_gas', UInt64), ('chain_id', UInt64), ('transaction_index', UInt64), ('transaction_hash', String), ('nonce', UInt64), ('from_address', String), ('to_address', String), ('value_binary', String), ('value_string', String), ('value_f64', Float64), ('input', String), ('gas_limit', UInt64), ('gas_used', UInt64), ('gas_price', UInt64), ('transaction_type', UInt32), ('max_priority_fee_per_gas', UInt64), ('max_fee_per_gas', UInt64), ('success', Boolean), ('n_input_bytes', UInt32), ('n_input_zero_bytes', UInt32), ('n_input_nonzero_bytes', UInt32), ('num_event_logs', UInt32), ('num_app_event_logs', UInt32), ('timestamp_dt', Datetime(time_unit='us', time_zone=None)), ('timestamp_date', Datetime(time_unit='us', time_zone=None))])
<class 'polars.dataframe.frame.DataFrame'>
num blocks: 31531


In [18]:
# Assuming you have a DataFrame named 'joined_df' with the required columns

if stack == 'op':
        l2_gas_used = pl.col("gas_used")
        l1_gas_used = pl.col("l1_gas_used")
elif stack == 'arb':
        # Need to add columns to cryo
        l2_gas_used = pl.col("gas_used") - pl.col("gas_used_for_l1")
        l1_gas_used = pl.col("gas_used_for_l1")
else:
        l2_gas_used = pl.col("gas_used")
        l1_gas_used = pl.col("gas_used") - pl.col("gas_used")

result_df = joined_df.group_by([pl.col("timestamp_date"), pl.col("chain_id")]).agg( # .filter( (pl.col("timestamp_dt") >= start_timestamp) & (pl.col("timestamp_dt") < end_timestamp) ) \
                num_blocks=pl.col("block_number").n_unique(),
                min_block_number=pl.col("block_number").min(),
                max_block_number=pl.col("block_number").max(),
                min_block_time=pl.col("timestamp").min(),
                max_block_time=pl.col("timestamp").max(),

                num_user_transactions=
                pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
                num_success_user_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
                num_success_event_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
                num_success_qualified_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_app_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
                num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

                total_gas_used=pl.col("gas_used").sum(),
                user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
                total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
                user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),

                # l2_fees_base_fees_eth=( (pl.col("gas_price") - pl.col("max_priority_fee_per_gas")) * pl.col("gas_used")).sum() / 1e18,
                # l2_fees_priority_fees_eth=pl.when(pl.col("gas_price") > 0).then(pl.col("max_priority_fee_per_gas") * pl.col("gas_used")).sum() / 1e18,
                # l2_fees_total_fees_eth=(pl.col("gas_price") * pl.col("gas_used")).sum() / 1e18,

                avg_l2_base_fee_gwei=( (pl.col("gas_price") - pl.col("max_priority_fee_per_gas")) * pl.col("gas_used")).sum()
                                        / pl.col("gas_used").sum() /1e9,
                avg_l2_priority_fee_gwei=pl.when(pl.col("gas_price") > 0).then((pl.col("max_priority_fee_per_gas")) * pl.col("gas_used")).sum()
                                        / pl.col("gas_used").sum() /1e9,
        )
# timestamp formats
# result_df = result_df.with_columns([
#     pl.date_from_epoch_seconds(pl.col("min_block_time")).alias("min_block_time_dt"),
#     pl.date_from_epoch_seconds(pl.col("max_block_time")).alias("max_block_time_dt")
# ])

result_df = result_df.to_pandas()

In [19]:
result_df.to_csv(f"data_outputs/{chain_name}_{start_timestamp}_{end_timestamp}.csv")

In [20]:
# Filter
# result_df = result_df[ (result_df['min_block_time']>= start_timestamp ) & (result_df['min_block_time']<= end_timestamp ) ]
#seems like 1 block before gets pulled. yolo.

In [21]:
result_df['min_block_time_dt'] = pd.to_datetime(result_df['min_block_time'], unit='s')
result_df['max_block_time_dt'] = pd.to_datetime(result_df['max_block_time'], unit='s')
display(result_df.sort_values(by='timestamp_date',ascending=False))

,timestamp_date,chain_id,num_blocks,min_block_number,max_block_number,min_block_time,max_block_time,num_user_transactions,num_success_user_transactions,num_success_event_transactions,num_success_qualified_transactions,num_senders,total_gas_used,user_gas_used,total_gas_used_per_block,user_gas_used_per_block,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,min_block_time_dt,max_block_time_dt
0,2024-05-10,666666666,3656,14579710,14583365,1715299200,1715300140,30962,23685,20283,20280,10753,6720598380,6720598380,1.838238e+06,1.838238e+06,0.031058,0.647707,2024-05-10 00:00:00,2024-05-10 00:15:40
1,2024-05-09,666666666,27875,14551835,14579709,1715292000,1715299199,229928,174452,149150,149128,74012,50782680656,50782680656,1.821800e+06,1.821800e+06,0.023474,0.284506,2024-05-09 22:00:00,2024-05-09 23:59:59


In [27]:
top_usage = joined_df.group_by([pl.col("from_address"), pl.col("to_address"), pl.col("chain_id")]).agg(
        num_blocks=pl.col("block_number").n_unique(),
                min_block_number=pl.col("block_number").min(),
                max_block_number=pl.col("block_number").max(),
                min_block_time=pl.col("timestamp").min(),
                max_block_time=pl.col("timestamp").max(),
        num_user_transactions=
                pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
        num_success_user_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
        num_success_event_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
        num_success_qualified_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_app_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
        num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

        total_gas_used=pl.col("gas_used").sum(),
        user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
        total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
        user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),
)
top_usage = top_usage.to_pandas()
top_usage = top_usage.sort_values(by = 'num_success_user_transactions', ascending=False)
print(top_usage.head(10))
top_usage.to_csv(f"data_outputs/top_usage_{chain_name}_{start_timestamp}_{end_timestamp}.csv")

                                      from_address  \
112947  0x030eef5e64d71ca3022dd0f37443a7b5e8222222   
62536   0x6b16be825b84d9a61b5ae370ea75dcd537555555   
111069  0xe68ca824c376ea70c439fd7f3c978772903e7f9d   
64117   0x50588be41da26e59c8eb1cbc83fda12b8a52cbc8   
68611   0xec65cf0c51d840bab2bbc39618bdf2f78bf0cc94   
49210   0x339d413ccefd986b1b3647a9cfa9cbbe70a30749   
111989  0x7fb6c9b61aff472e6b704ec107239a91c9ee86f4   
137272  0x7b10eba4e72cdcbc7e1856e7e9cee4554543b163   
33922   0x355f523dca6477792b8869731c522dbc4e3c7b2a   
41550   0x81f8404cd82f41ab2afd27920bccf5cb1b8caa61   

                                        to_address   chain_id  num_blocks  \
112947  0x6ac1c44f2e0a79a682dd6bc4a4267673b2167677  666666666       14211   
62536   0x6ac1c44f2e0a79a682dd6bc4a4267673b2167677  666666666       14200   
111069  0x831f011b38fd707229b2d1fcf3c8a1964200c9fe  666666666         962   
64117   0x2a3b6469f084fe592bc50eeabc148631adacb92e  666666666         135   
68611   0x6dce9f6579

In [25]:
top_contracts = joined_df.group_by([pl.col("to_address"), pl.col("chain_id")]).agg(
        num_blocks=pl.col("block_number").n_unique(),
                min_block_number=pl.col("block_number").min(),
                max_block_number=pl.col("block_number").max(),
                min_block_time=pl.col("timestamp").min(),
                max_block_time=pl.col("timestamp").max(),
        num_user_transactions=
                pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
        num_success_user_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
        num_success_event_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
        num_success_qualified_transactions=
                pl.when((pl.col("gas_price") > 0) & pl.col("success") & (pl.col("num_app_event_logs") > 0)).then(pl.col("transaction_hash")).count(),
        num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

        total_gas_used=pl.col("gas_used").sum(),
        user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
        total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
        user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),
)
top_contracts = top_contracts.to_pandas()
top_contracts = top_contracts.sort_values(by = 'num_success_user_transactions', ascending=False)
print(top_contracts.head(10))
top_contracts.to_csv(f"data_outputs/top_contracts_{chain_name}_{start_timestamp}_{end_timestamp}.csv")

                                     to_address   chain_id  \
19   0x5dae94e149cf2112ec625d46670047814aa9ac2a  666666666   
18   0xefc52ff373b19ce2164cea71b098a776969cd2a9  666666666   
41   0x6ac1c44f2e0a79a682dd6bc4a4267673b2167677  666666666   
3    0x831f011b38fd707229b2d1fcf3c8a1964200c9fe  666666666   
5    0x5a8e4e0dd630395b5afb8d3ac5b3ef269f0c8356  666666666   
88   0x6fb0f96bb2dcd32388ebbb6b13608928ed538218  666666666   
26   0x2a3b6469f084fe592bc50eeabc148631adacb92e  666666666   
14   0x3c2269811836af69497e5f486a85d7316753cf62  666666666   
54   0x6dce9f6579ad1d256a4233ac775d095174e3d720  666666666   
58   0x31834c4f583b9c0565eacb28fac24c1df97cb884  666666666   
63   0x7a1dbb83dbb1d4e3e692455533ba18f2fdf19dc9  666666666   
91   0x053cd976a539cc885dd141be360635fe9d259714  666666666   
78   0x9c0df4b950ca19db6fec13ab79ad180a9c15a41e  666666666   
36   0x5ff137d4b0fdcd49dca30c7cf57e578a026d2789  666666666   
97   0x000000000000000000000000000000000000006e  666666666   
106  0xe